In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

naw


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [3]:
pgd.search_name("platy")

,name,id
0,Monotremata,15991
1,Palaeoplatycerus,52389
2,Platymantis guentheri,41154
3,Australoplatyola,63651
4,Platydracus (Nesiolinus),92538
5,Platymantis sierramadrensis,50226
6,Platymantis boulengeri,41148
7,Platypsaris,84077
8,Platynus megalops,67683
9,Platycholeus,29482


In [4]:
pgd.fetch_tol_data(6451)

fetching http://tolweb.org/onlinecontributors/app?service=external&page=xml/TreeStructureService&node_id=6451 
Fetched 54 lines. Parsing and assigning...
All done. 5 organisms fetched.


### 2) load from raw file

In [5]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [6]:
#pgd.write_prep_data("data/primate_df.csv", "data/primate_links.txt")

### 3) load from prepared files

In [7]:
#pgd.load_prep_data("data/primate_df.csv", "data/primate_links.txt")

### return data for plotting

In [8]:
df, links_list = pgd.return_data()

## Plot Data

In [9]:
pgp = ph.phyloGraph(df, links_list)

In [10]:
#pgp.search_name('rrh')


In [11]:
pgp.create_plot_data(max_depth=30, max_nodes=50000)

/Users/Seth/Documents/tolGraph/phyloGraph.py:338: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting 6451


In [12]:
pgp.render_plot()
#pgp.render_plot(publish=True)

In [13]:
pgp.open_plot()